In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('data.csv')

x_train, y_train = dataset['time_before_failure'], dataset.iloc[:, dataset.columns != 'time_before_failure']

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [4]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 192.3 kB/s eta 0:00:0000:0100:23


In [5]:
!pip install optuna scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 KB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.3 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 KB 10.3 MB/s eta 0:00:00a 0:00:01


In [6]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error

def objective(trial):
    param = {
        'verbosity': 0,
        'objective': 'reg:squarederror',  # Задача регрессии
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1e-1, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1e-1, log=True),
    }

    model = xgb.XGBRegressor(**param)
    
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='neg_mean_absolute_error').mean()
    
    return score

In [7]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50, timeout=600)

[I 2024-08-27 13:15:28,330] A new study created in memory with name: no-name-671b51bf-3932-41ed-b425-bc9bce819fbc
[W 2024-08-27 13:15:28,332] Trial 0 failed with parameters: {} because of the following error: TypeError("objective() missing 2 required positional arguments: 'x_train' and 'y_train'").
Traceback (most recent call last):
  File "/home/galex/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
TypeError: objective() missing 2 required positional arguments: 'x_train' and 'y_train'
[W 2024-08-27 13:15:28,334] Trial 0 failed with value None.


TypeError: objective() missing 2 required positional arguments: 'x_train' and 'y_train'